In [1]:
import os
gpu_num = 0 # 使用 "" 来启用 CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sionna
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import mitsuba as mi

from sionna.mimo import StreamManagement

from sionna.ofdm import ResourceGrid, ResourceGridMapper, LSChannelEstimator, LMMSEEqualizer
from sionna.ofdm import OFDMModulator, OFDMDemodulator, ZFPrecoder, RemoveNulledSubcarriers

from sionna.channel.tr38901 import AntennaArray, CDL, Antenna
from sionna.channel import subcarrier_frequencies, cir_to_ofdm_channel, cir_to_time_channel, time_lag_discrete_time_channel
from sionna.channel import ApplyOFDMChannel, ApplyTimeChannel, OFDMChannel, TimeChannel

from sionna.fec.ldpc.encoding import LDPC5GEncoder
from sionna.fec.ldpc.decoding import LDPC5GDecoder

from sionna.mapping import Mapper, Demapper

from sionna.utils import BinarySource, ebnodb2no, sim_ber
from sionna.utils.metrics import compute_ber
# Import Sionna RT components
from mysionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

In [2]:
from sensing.scene import SensingScene
from sensing.target import SensingTarget as tg
tg1 = tg('human1','itu_wood')

In [3]:
scene = SensingScene("./Indoor/indoor.xml")

In [7]:
scene.scene.mi_scene.shapes()

[PLYMesh[
   name = "ceiling-itu_ceiling_board.ply",
   bbox = BoundingBox3f[
     min = [-3, -5, 3.5],
     max = [3, 5, 3.5]
   ],
   vertex_count = 4,
   vertices = [80 B of vertex data],
   face_count = 2,
   faces = [24 B of face data],
   face_normals = 1
 ],
 PLYMesh[
   name = "door-itu_plywood.ply",
   bbox = BoundingBox3f[
     min = [1.56861, 4.04366, -0.00357819],
     max = [3.04818, 4.51779, 3.14076]
   ],
   vertex_count = 14,
   vertices = [280 B of vertex data],
   face_count = 12,
   faces = [144 B of face data],
   face_normals = 1
 ],
 PLYMesh[
   name = "floor-itu_floorboard.ply",
   bbox = BoundingBox3f[
     min = [-3, -5, 0],
     max = [3, 5, 0]
   ],
   vertex_count = 4,
   vertices = [80 B of vertex data],
   face_count = 2,
   faces = [24 B of face data],
   face_normals = 1
 ],
 PLYMesh[
   name = "table1-itu_plywood.ply",
   bbox = BoundingBox3f[
     min = [-1.07471, -1.95782, 1.20652],
     max = [0.925293, 1.95782, 1.31491]
   ],
   vertex_count = 14,
 

In [8]:
tx_positions = np.array([[0,0,2.95],[0,-4.99,2.95]])
tx_look_dir = np.array([[0,0,0],[0,0,0]])
scene.set_tx(tx_positions,tx_look_dir)
map_center = [0,0,0.65]
map_size_x = 6
map_size_y = 10
cell_size = 0.1
cell_positions = scene.set_cell_positions(map_center,map_size_x,map_size_y,cell_size)

In [9]:
paths = scene.compute_paths(max_depth=4,num_samples=1000000,max_rx_each_turn=1000,scattering=False,diffraction=True)

In [11]:
paths[0].objects

<tf.Tensor: shape=(4, 1000, 2, 139), dtype=int32, numpy=
array([[[[-1,  3,  2, ...,  9, -1, -1],
         [-1,  8, 10, ..., -1, -1, -1]],

        [[-1,  3,  3, ...,  9, -1, -1],
         [-1, 11,  8, ..., -1, -1, -1]],

        [[-1,  3,  3, ...,  9, -1, -1],
         [-1, 11,  8, ..., -1, -1, -1]],

        ...,

        [[-1,  3,  3, ...,  9, 12, -1],
         [-1,  2,  9, ..., 30,  9, -1]],

        [[-1,  3,  3, ...,  9, 12, -1],
         [-1,  2,  9, ..., 30,  9, -1]],

        [[-1,  3,  3, ...,  9, 12, -1],
         [-1,  2,  8, ..., 30,  9, -1]]],


       [[[-1,  0,  9, ..., -1, -1, -1],
         [-1,  9, 11, ..., -1, -1, -1]],

        [[-1,  0,  0, ..., -1, -1, -1],
         [-1,  2,  9, ..., -1, -1, -1]],

        [[-1,  0,  0, ..., -1, -1, -1],
         [-1,  2,  9, ..., -1, -1, -1]],

        ...,

        [[-1,  0,  0, ..., -1, -1, -1],
         [-1,  9,  2, ..., -1, -1, -1]],

        [[-1,  0,  0, ..., -1, -1, -1],
         [-1,  9,  2, ..., -1, -1, -1]],

        [[-

In [ ]:
n_prims=0
primitives_2_objects = []
prim_offsets = []
mi_scene = scene.scene.mi_scene
for i,s in enumerate(mi_scene.shapes()):
    if not isinstance(s, mi.Mesh):
        raise ValueError('Only triangle meshes are supported')
    prim_offsets.append(n_prims)
    n_prims += s.face_count()
    primitives_2_objects += [i]*s.face_count()